This notebook can be used to sample the json files that serve as base for the hackaton

In [ ]:
import pandas as pd
import json
import numpy as np

# Prod schedule

In [ ]:
df_prod_sced = pd.DataFrame({"heat_sq":[1, 2], 
                             "head_id": ["heat-1", "heat-12"],
                            "steel_grade": "ST1",
                           "required_weight": np.random.binomial(2000, 0.5, size=2),
                            "chemistry": [{"cu_pct": 0.1}, {"cu_pct": 0.1}]})
df_prod_sced

In [ ]:
with open('production_schedule2.json', 'w') as outfile:
    json.dump(df_prod_sced.to_dict(orient='record'), outfile)

# Inventory

In [ ]:
df_inventory = pd.DataFrame({"scrap_type":["bushling", "pig_iron"], 
                             "required_weight": np.random.binomial(10000, 0.7, size=2),
                             "status": "on_hand"
                            })
df_inventory

In [ ]:
with open('srap_inventory2.json', 'w') as outfile:
    json.dump(df_inventory.to_dict(orient='record'), outfile)

# Orders

In [ ]:
df_order = pd.DataFrame({"scrap_type": ["bushling", "pig_iron"], 
                         "order_date":["20190405", "20190406"], 
                         "status": "delivered", 
                         "weight": np.random.binomial(10000, 0.7, size=2), 
                         "price_per_ton": np.random.binomial(200, 0.7, size=2)
                        })
df_order

In [ ]:
with open('srap_orders2.json', 'w') as outfile:
    json.dump(df_order.to_dict(orient='record'), outfile)